In [4]:
import os
import torch
from datetime import datetime
from denoising_diffusion_pytorch import Unet, GaussianDiffusion, Trainer

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
day_night = 'day' # choices 'night' , 'day'

In [7]:
date_time = datetime.now().strftime("%Y_%m_%d_%H_%M")
train_num_steps = 70000
timesteps = 1000
path_to_data = f'../../data/separated_night_day/{day_night}/train'

# Dataloader

In [8]:
import os
import random
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms


class DayNightDataset(Dataset):
    """
    Loads 64×64 RGB images from
        ../../data/separated_night_day/{day_night}/train
    or any similar folder that just contains .png/.jpg files.
    """
    _valid_ext = ('.png', '.jpg', '.jpeg', '.bmp')

    def __init__(self, root_dir, transform=None):
        self.root_dir  = root_dir
        self.transform = transform or transforms.ToTensor()

        self.image_paths = sorted(
            p for p in os.listdir(root_dir)
            if p.lower().endswith(self._valid_ext)
        )
        if not self.image_paths:
            raise RuntimeError(f'No images found in {root_dir}')

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_paths[idx])
        img = Image.open(img_path).convert('RGB')      # keep 3 channels
        return self.transform(img)                     # → [3,64,64] float in [0,1]


def get_daynight_loaders(day_night,
                         data_root='../../data/separated_night_day',
                         batch_size=32,
                         val_split=0.1,
                         num_workers=4,
                         pin_memory=False,
                         seed=None,
                         transform=None):
    """
    Returns (train_loader, val_loader) for either 'day' or 'night'.
    """

    path_to_data = os.path.join(data_root, day_night, 'train')
    dataset = DayNightDataset(path_to_data,
                              transform=transform or transforms.ToTensor())

    # ── train / val split ────────────────────────────────────────────
    val_len = int(len(dataset) * val_split)
    train_len = len(dataset) - val_len
    if seed is not None:
        generator = torch.Generator().manual_seed(seed)
        train_ds, val_ds = random_split(dataset, [train_len, val_len],
                                        generator=generator)
    else:
        train_ds, val_ds = random_split(dataset, [train_len, val_len])

    kwargs = dict(num_workers=num_workers, pin_memory=pin_memory)

    train_loader = DataLoader(train_ds,
                              batch_size=batch_size,
                              shuffle=True,
                              **kwargs)
    val_loader   = DataLoader(val_ds,
                              batch_size=batch_size,
                              shuffle=False,
                              **kwargs)

    return train_loader, val_loader


# Train DDPM

In [9]:
mults = (1, 2, 4)
mults_str = '_'.join(str(m) for m in mults)

In [10]:
# example: night‑only training set
train_loader, val_loader = get_daynight_loaders(
    day_night=day_night,
    batch_size=64,
    seed=42,
    transform=transforms.Compose([
        transforms.ToTensor(),          # scale to [0,1]
        # add your own flips / colour jitter here if you like
    ])
)

In [ ]:
model = Unet(
    dim = 64,
    dim_mults =mults, 
    flash_attn = True
)

diffusion = GaussianDiffusion(
    model,
    image_size = 64,
    timesteps = timesteps,           # number of steps
    sampling_timesteps = 250    # number of sampling timesteps (using ddim for faster inference [see citation for ddim paper])
)
results_folder = f'./results/{day_night}/{mults_str}/tr_stp_{train_num_steps}_stp{timesteps}/{date_time}'
os.makedirs(results_folder, exist_ok=True)
trainer = Trainer(
    diffusion,
    path_to_data,
    train_batch_size = 32,
    train_lr = 8e-5,
    train_num_steps = train_num_steps,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True,                       # turn on mixed precision
    calculate_fid = False,              # whether to calculate fid during training
    results_folder = results_folder,
    save_and_sample_every = train_num_steps//10,
    dl = train_loader,
)

trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


  0%|          | 0/70000 [00:00<?, ?it/s]/NAS/PROJECTS/BOMRGD/envs/ddpm_env/lib/python3.8/contextlib.py:83: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
loss: 0.0449:   2%|▏         | 1584/70000 [11:10<7:56:46,  2.39it/s] 

In [ ]:
day_night = 'night' # choices 'night' , 'day'
date_time = datetime.now().strftime("%Y_%m_%d_%H_%M")
train_num_steps = 70000
timesteps = 1000
path_to_data = f'../../data/separated_night_day/{day_night}/train'
# example: night‑only training set
train_loader, val_loader = get_daynight_loaders(
    day_night=day_night,
    batch_size=64,
    seed=42,
    transform=transforms.Compose([
        transforms.ToTensor(),          # scale to [0,1]
        # add your own flips / colour jitter here if you like
    ])
)
model = Unet(
    dim = 64,
    dim_mults =mults, 
    flash_attn = True
)

diffusion = GaussianDiffusion(
    model,
    image_size = 64,
    timesteps = timesteps,           # number of steps
    sampling_timesteps = 250    # number of sampling timesteps (using ddim for faster inference [see citation for ddim paper])
)
results_folder = f'./results/{day_night}/{mults_str}/tr_stp_{train_num_steps}_stp{timesteps}/{date_time}'
os.makedirs(results_folder, exist_ok=True)
trainer = Trainer(
    diffusion,
    path_to_data,
    train_batch_size = 32,
    train_lr = 8e-5,
    train_num_steps = train_num_steps,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True,                       # turn on mixed precision
    calculate_fid = False,              # whether to calculate fid during training
    results_folder = results_folder,
    save_and_sample_every = train_num_steps//10,
    dl = train_loader,
)

trainer.train()